In [1]:
# sensitivity analysis
# Dash Kamriani Beard 07/01/2024
# Lawrence Berkeley National Lab
import subprocess
import re

# SET THESE TO YOUR INPUT FILE PATHS
chemical_inp = "./chemical.inp"
solute_inp = "./solute.inp"
flow_inp = "./flow.inp"
simulation_executable = "./treactv4159omp_eco2n_macos"

# TEMPERATURE RANGE
temp_start = 0
temp_stop = 500
temp_step = 10


# utility functions

def runsim():
    subprocess.check_call([simulation_executable])
    return

def read_file(path):
    try:
        with open(path, 'r') as file:
            return file.read()
    except FileNotFoundError:
        print(f"The file at {path} was not found.")
    except IOError:
        print(f"An error occurred while reading the file at {path}.")

def split_text(path):
    text = read_file(path).splitlines()
    return text
    
def search_2d_array(array, target):
    flag = 0
    out = []
    for index, text in enumerate(array):
        if flag == 1:
            out.append([index, text])
            flag = 0
        if target in text:
            flag = 1
    return out

def find_second_non_space(arr):
    non_space_count = 0

    for index, element in enumerate(arr):
        if element.strip():  # Checks if the element is not empty or whitespace
            non_space_count += 1
            if non_space_count == 2:
                return [index, element]

    return None, None


#need to figure out a way to quantify what simulation results are, could generate plots and list them, or calculate %mineralized over time 
def make_plot(inp, result):
    pass
    return

In [3]:
# designed for chemical.inp (for now)
def vary_temp(path):

    # put array into a structure -> text[lines][words]
    text = []
    for line in split_text(path):
        words = re.findall(r'\S+|\s+', line)
        text.append(words)
        
    # find keyword "Speciation" and copy the NEXT row into temperature_row
    temperature_row = search_2d_array(text, "Speciation") 

    # build array of all temperature parameter locations within their respective lines
    temperature = []
    for i in temperature_row: 
        temperature.append(find_second_non_space(i[1])) 
        
    original_temp = text[temperature_row[0][0]][temperature[0][0]] # store the first original temperature

    # generate files with modified temperature parameter
    for num in range(temp_start, temp_stop + temp_step, temp_step):
        new_text = []
        for i in range(len(temperature_row)):
            text[temperature_row[i][0]][temperature[i][0]] = str(float(num)) # find and replace 
        path = f"chemical-{num}.inp"
        open(f"chemical-{num}.inp", 'w').close()
        for i in range(len(text)):
            new_text.append(''.join(text[i]))
            with open(path, 'a') as file:
                file.write(new_text[i] + '\n')
    
    for i in range(len(temperature_row)):
            text[temperature_row[i][0]][temperature[i][0]] = original_temp # find and replace 
    return 

vary_temp(chemical_inp)


In [345]:
# # TODO: make functions for other parameters, figure out how to run simulations in parallel, figure out how to quantify data from simulations and make plots

# def vary_pressure(path):
#     text = []
#     for line in split_text(path):
#         words = re.findall(r'\S+|\s+', line)
#         text.append(words)
#     temperature_row = search_2d_array(text, "Speciation")
#     temperature = find_second_non_space(temperature_row[1])
#     original_temp = text[temperature_row[0]][temperature[0]]
    
#     for num in range(temp_start, temp_stop + temp_step, temp_step):
#         new_text = []
#         text[temperature_row[0]][temperature[0]] = str(float(num))
#         path = f"chemical-{num}.inp"
#         open(f"chemical-{num}.inp", 'w').close()
#         for i in range(len(text)):
#             new_text.append(''.join(text[i]))
#             with open(path, 'a') as file:
#                 file.write(new_text[i] + '\n')

#     text[temperature_row[0]][temperature[0]] = original_temp
#     return 